In [201]:
import lxml.etree as etree
import pandas as pd
import gzip

In [278]:
fname = "uspto/ipgb20080909.xl.gz"
parser = etree.XMLParser(remove_blank_text=True)

count = 0
validLineNumbers = []
numLines = 0
numDesigns = 0
citationTypes = []

df = None
with gzip.open(fname, 'rb') as fin:
    for (lineNumber, line) in enumerate(fin):
        numLines = numLines + 1
        try:
            root = etree.fromstring(line, parser)
        except etree.XMLSyntaxError:
            print("skipping due to parse error")
 
        applicationType = root.find("us-bibliographic-data-grant/application-reference").attrib.get("appl-type")
        if applicationType == 'design':
            numDesigns = numDesigns + 1
            continue
            
        # get basic data
        date = root.find("us-bibliographic-data-grant/publication-reference/document-id/date").text
        docNumber = root.find("us-bibliographic-data-grant/publication-reference/document-id/doc-number").text
        kind = root.find("us-bibliographic-data-grant/publication-reference/document-id/kind").text
        applicationType = root.find("us-bibliographic-data-grant/application-reference").attrib.get("appl-type")
        inventionTitle = root.find("us-bibliographic-data-grant/invention-title").text
        numberOfClaims = root.find("us-bibliographic-data-grant/number-of-claims").text

        # skip entries without abstracts
        abstract = []
        if root.find("abstract") is not None:
            abstract = etree.tostring(root.find("abstract"), encoding='UTF-8', method="text")
            #print(abstract[:20])
            validLineNumbers.append(lineNumber)
            
        referencesCited = root.find("us-bibliographic-data-grant/references-cited")
        
        examinerCitations = []
        applicantCitations = []         
        if referencesCited is not None:
            # remove non-patent citations
            referencesCited = referencesCited.getchildren()
            referencesCited = [citation for citation in referencesCited if citation.find("patcit") is not None]
            for citation in referencesCited:
                citationDocNum = citation.find("patcit/document-id/doc-number").text
                citationType = etree.tostring(citation.find("category"), method="text")
                citationTypes.append(citationType)
                if citationType == b'cited by examiner':
                    examinerCitations.append(citationDocNum)
                else:
                    applicantCitations.append(citationDocNum)
    
        print(len(examinerCitations))
        #add the information to the DataFrame
        if df is None:
            df = pd.DataFrame({'PatentNumber': [docNumber], 'examinerCitations': [examinerCitations], 'applicantCitations': [applicantCitations]})
        else:
            df.append(pd.DataFrame({'PatentNumber': [docNumber], 'examinerCitations': [examinerCitations], 'applicantCitations': [applicantCitations]}))


0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
3
0
1
0
0
0
0
0
1
0
0
29
20
1
6
4
15
17
26
30
2
2
3
9
33
5
0
6
10
6
4
2
6
2
7
4
8
5
17
8
3
9
7
5
6
4
7
8
6
3
7
14
3
5
4
13
14
2
1
4
6
6
3
3
4
4
6
4
5
5
2
38
44
23
9
5
8
21
11
11
12
5
5
4
5
9
0
2
8
0
2
16
5
11
3
7
7
8
8
4
22
9
15
11
10
13
27
70
3
6
17
11
6
27
38
9
12
18
1
14
5
3
11
4
5
5
4
9
21
3
2
1
0
14
2
3
5
10
7
6
2
4
9
2
7
5
5
8
5
7
16
24
16
21
7
1
7
5
6
3
8
2
3
9
2
3
2
6
0
6
3
14
3
9
10
4
5
23
11
6
6
4
3
3
6
4
3
4
2
9
14
1
2
8
9
9
7
4
9
10
3
5
9
1
7
10
5
4
6
7
11
7
11
7
8
7
5
14
18
24
15
4
3
3
2
5
3
4
3
1
6
5
4
1
10
5
10
4
8
5
7
7
23
4
2
9
9
7
8
5
1
7
4
6
2
3
18
16
0
0
5
10
18
24
9
1
3
5
5
1
17
2
6
8
1
3
1
1
7
0
3
8
20
4
9
10
10
3
3
11
6
16
1
0
1
7
6
7
8
5
2
5
4
6
1
1
6
1
12
11
7
5
6
3
12
0
13
4
13
16
5
8
4
9
5
4
11
14
5
13
5
4
6
8
6
10
10
1
6
1
5
6
3
5
1
2
3
2
11
39
5
6
6
1
5
13
9
15
18
0
7
8
4
8
8
9
4
2
6
6
17
26
15
5
8
23
3
6
5
17
9
2
8
3
10
8
6
8
14
7
3
10
5
1
8
3
5
8
3
11
0
10
8
8
6
12
3
8
31
9
1
4
8
11
13
7
12
3
13
5
5
18
8
4
14
13
1

In [282]:
len(df)

1

In [251]:
document = root.getchildren()[0]
fields = []
for item in document.iter():
    fields.append(item.tag)
fields = list(set(fields))
fields.sort() 
#etree.tostring(root.find("abstract"), method="text")
#print(fields)
#print(etree.tostring(root, pretty_print=True).decode('UTF-8'))

date = root.find("us-bibliographic-data-grant/publication-reference/document-id/date").text
docNumber = root.find("us-bibliographic-data-grant/publication-reference/document-id/doc-number").text
kind = root.find("us-bibliographic-data-grant/publication-reference/document-id/kind").text
applicationType = root.find("us-bibliographic-data-grant/application-reference").attrib.get("appl-type")
inventionTitle = root.find("us-bibliographic-data-grant/invention-title").text
numberOfClaims = root.find("us-bibliographic-data-grant/number-of-claims").text

referencesCited = root.find("us-bibliographic-data-grant/references-cited").getchildren()
# remove non-patent citations
referencesCited = [citation for citation in referencesCited if citation.find("patcit") is not None]

examinerCitations = []
applicantCitations = []
for citation in referencesCited:
    citationDocNum = citation.find("patcit/document-id/doc-number").text
    citationType = etree.tostring(citation.find("category"), method="text")
    if citationType == b'cited by examiner':
        examinerCitations.append(citationDocNum)
    else:
        applicantCitations.append(citationDocNum)
        
print(examinerCitations)
#[date, docNumber, kind, applicationType, inventionTitle, numberOfClaims]

['5390297', '5832208', '5987525', '6347375', '6385596', '6388957', '6496802', '6577735', '6594686', '6735699', '6766305', '6868405', '6868494', '6873975', '6915425', '7266202', '7349381', '2001/0041989', '2001/0051996', '2002/0023020', '2002/0129140', '2002/0152262']


In [238]:
etree.tostring(referencesCited[42].find("patcit"))

b'<patcit num="00043"><document-id><country>EP</country><doc-number>1 041 767</doc-number><date>20001000</date></document-id></patcit>'

In [245]:
print(etree.tostring(root.find("us-bibliographic-data-grant/references-cited"), pretty_print=True).decode('UTF-8'))

<references-cited>
  <citation>
    <patcit num="00001">
      <document-id>
        <country>US</country>
        <doc-number>4136395</doc-number>
        <kind>A</kind>
        <name>Kolpek et al.</name>
        <date>19790100</date>
      </document-id>
    </patcit>
    <category>cited by other</category>
  </citation>
  <citation>
    <patcit num="00002">
      <document-id>
        <country>US</country>
        <doc-number>5142662</doc-number>
        <kind>A</kind>
        <name>Gump et al.</name>
        <date>19920800</date>
      </document-id>
    </patcit>
    <category>cited by other</category>
  </citation>
  <citation>
    <patcit num="00003">
      <document-id>
        <country>US</country>
        <doc-number>5390297</doc-number>
        <kind>A</kind>
        <name>Barber et al.</name>
        <date>19950200</date>
      </document-id>
    </patcit>
    <category>cited by examiner</category>
    <classification-national>
      <country>US</country>
      <main-classi